In [1]:
import numpy as np
from testCases import *
from gc_utils import sigmoid,relu,dictionary_to_vector,vector_to_dictionary,gradients_to_vector

In [2]:
def forward_propagation(x,theta):
    J = theta*x
    return J


In [5]:
import requests

url = 'https://www.baidu.com/'
re = requests.get(url)
print(re.text)


<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css><title>ç¾åº¦ä¸ä¸ï¼ä½ å°±ç¥é</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt value maxlength=255 autocomplete=off autofocus=autofocus></span><sp

In [6]:
?requests.get()

In [3]:
x,theta=2,4
J = forward_propagation(x,theta)
print "J = "+str(J)

J = 8


In [7]:
1+1

2

In [4]:
def backward_propagation(x,theta):
    dtheta = x
    return dtheta


In [6]:
x,theta = 2,4
dtheta = backward_propagation(x,theta)
print "dtheta = "+str(dtheta)

dtheta = 2


In [25]:
def gradient_check(x,theta,epsilon=1e-7):
    thetaplus = theta+epsilon
    thetaminus = theta-epsilon
    J_plus = forward_propagation(x,thetaplus)
    J_minus = forward_propagation(x,thetaminus)
    gradapprox = (J_plus-J_minus)/(2*epsilon)
    grad = backward_propagation(x,theta)
    numerator = np.linalg.norm((grad-gradapprox))
    denominator = np.linalg.norm(grad)+np.linalg.norm(gradapprox)
    difference = numerator/denominator
    
    if difference <1e-7:
        print "The gradient is correct!"
    else:
        print "The gradient is wrong!"
    return difference
    

In [26]:
x,theta =2,4
difference = gradient_check(x,theta)
print "difference ="+str(difference)

The gradient is correct!
difference =2.91933588329e-10


In [31]:
def forward_propagation_n(X,Y,parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    Z1 = np.dot(W1,X)+b1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = relu(Z2)
    Z3 = np.dot(W3,A2)+b3
    A3 = sigmoid(Z3)
    
    logprobs = np.multiply(-np.log(A3),Y)+np.multiply(-np.log(1-A3),1-Y)
    cost = 1./m*np.sum(logprobs)
    cache = (Z1,A1,W1,b1,Z2,A2,W2,b2,Z3,A3,W3,b3)
    return cost,cache

In [32]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation presented in figure 2.
    
    Arguments:
    X -- input datapoint, of shape (input size, 1)
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [55]:
def gradient_check_n(parameters,gradients,X,Y,epsilon=1e-7):
    
    parameters_values,_ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters,1))
    J_minus = np.zeros((num_parameters,1))
    gradapprox = np.zeros((num_parameters,1))
    
    for i in range(num_parameters):
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0]+epsilon
        J_plus[i],_ = forward_propagation(X,Y,vector_to_dictionary(thetaplus))
        
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0]-epsilon
        J_minus[i],_ = forward_propagation(X,Y,vector_to_dictionary(thetaminus))
        
        gradapprox[i] = (J_plus[i]-J_minus[i])/(2*epsilon)
    numerator = np.linalg.norm((grad-gradapprox))
    denominator = np.linalg.norm(grad)+np.linalg.norm(gradapprox)
    difference = numerator/denominator
    
    if difference >2e-7:
        print "wrong:"+str(difference)
    else:
        print "correct:"+str(difference)
    return difference

        

In [56]:
X,Y,parameters = gradient_check_n_test_case()
cost,cache = forward_propagation_n(X,Y,parameters)
gradients = backward_propagation_n(X,Y,cache)
difference = gradient_check_n(parameters,gradients,X,Y)

wrong:0.285093156776


In [52]:
A = np.array([1,2,3,4])
B = np.copy(A)
print B


[1 2 3 4]
